In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import gaussian, threshold_otsu, threshold_multiotsu, sobel, threshold_sauvola
from skimage.morphology import remove_small_objects, disk, binary_closing
from scipy.ndimage import zoom, binary_dilation, binary_erosion, distance_transform_edt
from skimage.measure import label, regionprops
from skimage import io, exposure, color
from skimage import measure, morphology
from skimage import exposure
from czifile import imread
import cv2
import re
from matplotlib.ticker import MaxNLocator
from cellpose import models, plot 
model = models.Cellpose(model_type='cyto')

In [2]:
MIN_INCLUSION_SIZE = 10
MAX_INCLUSION_SIZE = 2000

In [3]:
def display_image(image, path, type):
    """Display the image."""
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{path} {type}")
    plt.show()

def extract_image_paths(folder):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

def read_image(image_path):
    """Read the LSM image from the specified path."""
    return imread(image_path)

def count(mask): 
    """Count the number of unique labels in the mask."""
    return len(np.unique(label(mask))) - 1  # Exclude background label (0)

def extract_channels(image: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Extract green and red channels from the squeezed image (shape: [Z, C, H, W]).""" 
    return image[0], image[1]

def preprocess_green_channel(green_channel):
    """
    Preprocess the green fluorescence channel for better segmentation and inclusion detection.
    - Applies Gaussian blur to reduce noise.
    - Enhances contrast using sigmoid adjustment.
    - Normalizes intensities to [0, 1] for consistent processing.
    """
    confocal_img = gaussian(green_channel, sigma=2)
    confocal_img = exposure.adjust_sigmoid(green_channel, cutoff=0.25)
    confocal_img = normalize_image(confocal_img)
    return confocal_img

    return circular_mask, non_circular_mask
def normalize_image(image):
    """
    Normalize the image to the range [0, 1].
    This is useful for consistent processing across different images.
    """
    return (image - np.min(image)) / (np.max(image) - np.min(image))

def calculate_surface_area(labeled_image: np.ndarray) -> float:
    """Calculate the total surface area for labeled regions."""
    props = regionprops(labeled_image)
    return sum(prop.area for prop in props)


def split_by_circularity(binary_mask: np.ndarray, threshold: float):
    """
    Splits objects in a binary mask based on circularity.
    
    Parameters:
        binary_mask (ndarray): Binary image with objects (1) and background (0).
        threshold (float): Circularity threshold (0 to 1). Higher is more circular.
        
    Returns:
        circular_mask (ndarray): Mask of objects above threshold.
        non_circular_mask (ndarray): Mask of objects below or equal to threshold.
    """
    labeled = label(binary_mask)
    circular_mask = np.zeros_like(binary_mask, dtype=bool)
    non_circular_mask = np.zeros_like(binary_mask, dtype=bool)

    for region in regionprops(labeled):

        perimeter = region.perimeter or 1  # Avoid divide-by-zero
        circularity = 4 * np.pi * region.area / (perimeter ** 2)

        if circularity > threshold:
            circular_mask[labeled == region.label] = True
        else:
            non_circular_mask[labeled == region.label] = True

    return circular_mask, non_circular_mask

def segment_cells(green_channel):
    """
    Segment whole cells in the green channel using Cellpose.
    - Normalizes image intensity.
    - Suppresses bright spots (e.g., inclusions) to better detect cell boundaries.
    - Applies Gaussian blur for smoother segmentation input.
    - Gradually increases segmentation diameter until at least one cell is detected.
    """
    green_channel = normalize_image(green_channel)
    percentile_99 = np.percentile(green_channel, 99)
    
    # Suppress very bright pixels (inclusions)
    green_channel_remove_inclusions = np.where(green_channel < percentile_99, green_channel, percentile_99)
    green_channel_remove_inclusions = gaussian(green_channel_remove_inclusions, sigma=5)

    # Normalize again after processing
    green_channel_remove_inclusions = normalize_image(green_channel_remove_inclusions)

    # Try different diameters until cells are detected
    diameter = 150
    while diameter < 500:
        masks, flows, styles, diams = model.eval(green_channel_remove_inclusions, diameter=diameter, channels=[0, 0])
        labeled_cells = label(masks)
        if np.max(labeled_cells) > 0:
            return labeled_cells
        diameter += 25

    # No cells found
    return None

def extract_inclusions(green_channel, mask, display_graph=False):
    """
    Extract potential inclusions inside a cell.
    - Blurs and masks the cell region.
    - Computes intensity statistics for thresholding.
    - Applies different threshold strategies depending on intensity distribution.
    - Removes objects that are too small or too large to be inclusions.
    - Optionally shows histogram for debugging.
    """
    applied_mask_blurred = gaussian(green_channel, sigma=1) * mask
    applied_mask_eliminate_background = applied_mask_blurred[applied_mask_blurred > 0]

    # Normalize the signal within the masked region
    applied_mask_eliminate_background = normalize_image(applied_mask_eliminate_background)


    # Compute descriptive statistics for intensity distribution
    q3 = np.percentile(applied_mask_eliminate_background, 75)
    hist, bin_edges = np.histogram(applied_mask_eliminate_background, bins='fd')
    applied_mask = normalize_image(green_channel) * mask


    # Decide on thresholding strategy based on upper quartile
    if q3 < 0.4 and len(bin_edges) > 20:
        #threshold = max(threshold_otsu(applied_mask), 0.5)
        threshold = max(threshold_otsu(applied_mask), 0.5)
    else:
        threshold = 0.95

    # Apply threshold and size-based filters
    #threshold = max(threshold_otsu(applied_mask), 0.38)
    inclusions = applied_mask > threshold
    inclusions = remove_small_objects(inclusions, min_size=MIN_INCLUSION_SIZE)
    #inclusions = inclusions ^ remove_small_objects(inclusions, min_size=MAX_INCLUSION_SIZE)


    # Optional histogram display
    if display_graph:
        print("Threshold: ", threshold)
        print("Bin count", len(bin_edges))
        plt.hist(applied_mask_eliminate_background, bins='fd')
        plt.axvline(q3, color='purple', linestyle='dashed', linewidth=2, label=f'Q3: {q3:.2f}')
        plt.legend()
        plt.title("Intensity histogram")
        plt.show()


    return inclusions

def generate_inclusion_image(green_channel, labeled_cells):
    """
    Generate a binary image with all inclusions from all cells.
    - Loops through each segmented cell.
    - Extracts inclusions from each cell region.
    - Combines all into one final binary image.
    """
    inclusion_image = np.zeros_like(green_channel)

    for i, cell in enumerate(regionprops(labeled_cells)):
        if cell.area < 100:
            continue
        mask = labeled_cells == cell.label
        inclusions = extract_inclusions(green_channel, mask)
        inclusion_image += inclusions  # adds binary inclusion mask

    return inclusion_image


def filter_objects_by_overlap(source_mask, target_mask):
    """
    Keeps objects in `source_mask` that overlap with `target_mask`.

    Parameters:
        source_mask (ndarray): Binary mask with objects to test (e.g. clustered mitochondria).
        target_mask (ndarray): Binary mask defining inclusion region.

    Returns:
        filtered_mask (ndarray): Binary mask of source objects overlapping with target.
    """
    labeled_source = label(source_mask)
    filtered_mask = np.zeros_like(source_mask, dtype=bool)

    for region in regionprops(labeled_source):
        coords = tuple(region.coords.T)
        if np.any(target_mask[coords]):
            filtered_mask[coords] = True

    return filtered_mask

def find_swiss_cheese_inclusions(inclusion_image, red_channel_thresholded, verbose=False):
    swiss_chess_inclusions = np.zeros_like(inclusion_image)
    regular_inclusions = np.zeros_like(inclusion_image)
    labeled_inclusions = label(inclusion_image)
    for i, inclusion in enumerate(regionprops(labeled_inclusions)):
        mask = labeled_inclusions == inclusion.label
        overlap = mask * red_channel_thresholded
        if np.sum(overlap) > 0:
            swiss_chess_inclusions += mask
        else:
            regular_inclusions += mask
    return swiss_chess_inclusions, regular_inclusions

def seperate_single_LDS(inclusion_image, single_LDS_mask):
    single_LDS_in_inclusions = np.zeros_like(single_LDS_mask)
    single_LDS_in_cyto = np.zeros_like(single_LDS_mask)

    labled_LDS = label(single_LDS_mask)
    for i, region in enumerate(regionprops(labled_LDS)):
        mask = labled_LDS == region.label
        overlap = mask * inclusion_image
        if np.sum(overlap) > 0:
            single_LDS_in_inclusions += mask
        else:
            single_LDS_in_cyto += mask
    return single_LDS_in_inclusions, single_LDS_in_cyto


In [4]:
def analysis(red: np.ndarray, green:np.ndarray, path:str) -> pd.DataFrame:
    data = []
    df_cell_summary = pd.DataFrame()

    print("Starting analysis...")

    # Preprocess the green channel
    green = preprocess_green_channel(green)

    
    #display_image(green, path, "Green Channel")

    #return green

    labeled_cells = segment_cells(green)
    #display_image(labeled_cells, path, "Labeled Cells")

    inclusion_image = generate_inclusion_image(green, labeled_cells)

    #display_image(inclusion_image, path, "Inclusion Image")


    contrast_adjusted_red_normalized = (red - red.min()) / (red.max() - red.min())
    threshold_value_mitochondria = np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 3) #1.75 works well
    mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria
    mitochondria_thresholded = remove_small_objects(mitochondria_thresholded, min_size=10)


    swiss_cheese_inclusions, regular_inclusions = find_swiss_cheese_inclusions(inclusion_image, mitochondria_thresholded)
    #display_image(swiss_cheese_inclusions, path, "Swiss Cheese Inclusions")
    #display_image(regular_inclusions, path, "Regular Inclusions")

    #display_image(red, path, "Red Channel")
    #display_image(mitochondria_thresholded, path, "Mitochondria Thresholded")

    single, clustered = split_by_circularity(mitochondria_thresholded, 0.7)

    #display_image(single, path, "Single Mitochondria")
    #display_image(clustered, path, "Clustered Mitochondria")
    cell_count = 1
    for i, cell in enumerate(regionprops(labeled_cells)):
        if cell.area < 100:  # Skip tiny regions likely to be noise
            continue

                # Create a mask for the current cell
        mask = labeled_cells == cell.label

        original_cell = green * mask

        inclusions_in_cell = inclusion_image * mask
        swiss_cheese_inclusions_in_cell = swiss_cheese_inclusions * mask
        regular_inclusions_in_cell = regular_inclusions * mask
        mitochondria_in_cell = mitochondria_thresholded * mask
        single_mitochondria_in_cell = single * mask
        clustered_mitochondria_in_cell = clustered * mask


        # single LDS in inclusion 
        single_ld_in_inclusion, single_ld_in_cyto = seperate_single_LDS(inclusions_in_cell, single_mitochondria_in_cell)
        # clustered LDS in inclusion

        
        clustered_ld_in_inclusion = filter_objects_by_overlap(clustered_mitochondria_in_cell, inclusions_in_cell)

 

        # Calculate average size of inclusions
        if count(inclusions_in_cell) > 0:
            average_inclusion_size = np.sum(inclusions_in_cell) / count(inclusions_in_cell)
        else:
            average_inclusion_size = 0


        data.append({
            'Image Path': path,
            'Cell Number': cell_count,
            'Cell Area': cell.area,
            'Number of Inclusions': count(inclusions_in_cell), 
            'Number of Swiss Cheese Inclusions': count(swiss_cheese_inclusions_in_cell),  
            'Number of Solid Inclusions': count(regular_inclusions_in_cell),  
            'Average Size of Inclusion': average_inclusion_size,
            'Single LDS in Inclusion': count(single_ld_in_inclusion),
            'Single LDS in Cyto': count(single_ld_in_cyto),
            'Total Single LDS in Cell': count(single_mitochondria_in_cell),
            'Clustered LDS in Inclusion': count(clustered_ld_in_inclusion),
            'Clustered LDS in Cyto': count(clustered_mitochondria_in_cell) - count(clustered_ld_in_inclusion),
            'Total Clustered LDS in Cell': count(clustered_mitochondria_in_cell),
            'Total LDS in Cell': count(mitochondria_in_cell),
        })
        cell_count += 1

    df_cell_summary = pd.DataFrame(data)
    return df_cell_summary
    





In [5]:
def main(image_folder):
    images_to_analyze = extract_image_paths(image_folder)
    output_dir = os.getcwd()
    df_cell_summary_list = []

    for path in images_to_analyze:
        image = read_image(path)
        image_squeezed = np.squeeze(image) 
    
        red, green = extract_channels(image_squeezed)

        df_cell_summary = analysis(red, green, path)
        df_cell_summary_list.append(df_cell_summary)

    combined_cell_summary_df = pd.concat(df_cell_summary_list, ignore_index=True)
    output_summary_path = os.path.join(output_dir, '061725_SUMMARY.xlsx')
    combined_cell_summary_df.to_excel(output_summary_path, index=False)

if __name__ == "__main__":
    image_folder = '061725_images'
    main(image_folder)

    

Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting analysis...
Starting anal